### Example: US States Data
    1. Merge and join operations come up most often when one is combining data from different sources.
    2. Here we will consider an example of some data about US states and their populations. 
    3. The data files can be found at http://github.com/jakevdp/dataUSstates/

In [37]:
import numpy as np
import pandas as pd

In [38]:
# Reading state-population.csv file
state_population = pd.read_csv('state-population.csv')
state_population.head()

state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

In [39]:
# Reading state-areas.csv file
state_areas = pd.read_csv('state-areas.csv')
state_areas.head()

state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

In [40]:
# Reading state-abbrevs.csv file
state_abbrevs = pd.read_csv('state-abbrevs.csv')
state_abbrevs.head()

state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

### Problem Statement:
    Given this information, say we want to compute a relatively straightforward result:
    Rank US states and territories by their 2010 population density.
    We clearly have the data here to find this result, but we’ll have to combine the datasets to get it.

In [41]:
# Step 1 : We’ll start with a many-to-one merge that will give us the full state name within the population DataFrame
# Step 2 : We want to merge based on the state/region column of pop , and the abbreviation column of abbrevs .
# Step3 : We’ll use how='outer' to make sure no data is thrown away due to mismatched labels.
merged = pd.merge(state_population, state_abbrevs, how = 'outer', left_on = 'state/region', right_on = 'abbreviation')
merged = merged.drop('abbreviation', axis = 1)
merged.head()

state/region     ages  year  population    state
0           AL  under18  2012   1117489.0  Alabama
1           AL    total  2012   4817528.0  Alabama
2           AL  under18  2010   1130966.0  Alabama
3           AL    total  2010   4785570.0  Alabama
4           AL  under18  2011   1125763.0  Alabama

In [42]:
# Let’s double-check whether there were any mismatches here, which we can do by looking for rows with nulls:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [43]:
merged[merged['population'].isnull()]

state/region     ages  year  population state
2448           PR  under18  1990         NaN   NaN
2449           PR    total  1990         NaN   NaN
2450           PR    total  1991         NaN   NaN
2451           PR  under18  1991         NaN   NaN
2452           PR    total  1993         NaN   NaN
2453           PR  under18  1993         NaN   NaN
2454           PR  under18  1992         NaN   NaN
2455           PR    total  1992         NaN   NaN
2456           PR  under18  1994         NaN   NaN
2457           PR    total  1994         NaN   NaN
2458           PR    total  1995         NaN   NaN
2459           PR  under18  1995         NaN   NaN
2460           PR  under18  1996         NaN   NaN
2461           PR    total  1996         NaN   NaN
2462           PR  under18  1998         NaN   NaN
2463           PR    total  1998         NaN   NaN
2464           PR    total  1997         NaN   NaN
2465           PR  under18  1997         NaN   NaN
2466           PR    total  1999         NaN   NaN
2467           PR  under18  1999         NaN   NaN

In [44]:
# we see also that some of the new state entries are also null, which means that there was no corresponding entry in the abbrevs key! Let’s figure out which regions lack this match
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [45]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [46]:
# Now we can merge the result with the area data using a similar procedure
final = pd.merge(merged, state_areas, on = 'state', how = 'left')
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0

In [47]:
# let’s check for nulls to see if there were any mismatches
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [48]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [49]:
# We see that our areas DataFrame does not contain the area of the United States as a whole.
# we’ll just drop the null values because the population density of the entire United States is not relevant to our current discussion.
final.dropna(inplace=True)
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0

In [64]:
# Now we have all the data we need. To answer the question of interest, let’s first select the portion of the data corresponding with the year 2000, and the total population.
# We’ll use the query() function to do this quickly.
data2010 = final.query('year == 2010 & ages == "total"')
# Now let’s compute the population density and display it in order.
# We’ll start by reindexing our data on the state, and then compute the result.
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending = False, inplace = True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [65]:
# We can also check the end of the list:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64